In [1]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from unstructured.partition.pdf import partition_pdf

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:

output_path = "/workspaces/AgenticAI-Systems/RAG/Document/"
file_path = output_path + 'مذكرة دفاع مقدمة من مكتبنا جلسة  27-6-2019.pdf'

# Reference: https://docs.unstructured.io/open-source/core-functionality/chunking
chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables
 
    extract_image_block_types=["Image","Table","Text"],   # Add 'Table' to list to extract image of tables
    image_output_dir_path=output_path,   # if None, images and tables will saved in base64
 
    extract_image_block_to_payload=True,   # if true, will extract base64 for API usage
 
    chunking_strategy="by_title",          # or 'basic'
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,       # defaults to 0
    new_after_n_chars=6000,
 
    # extract_images_in_pdf=True,          # deprecated
)

In [17]:
import os
os.environ["GROQ_API_KEY"] = "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"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "True"

In [18]:
len(chunks)

2

In [19]:
chunks[0].to_dict()

{'type': 'CompositeElement',
 'element_id': 'd142fd3b1c52141ef389e0d0e2a4122e',
 'text': 'ol Crom tt AU us ewe aall Gas. lS pitty (gumial! Lam ela 3S Uggle (ou GHA Sait ats gs As a: ye Aas ia gous Pig dane Slew aitase alls / duull—4\n\nVe VA/A/VV Meade Ua shall 9 £/ IS gtlae Ye VA/VAVY aby ce geall (8 4ysqll gh / sud 5 Agel) ol) die als\n\nei\n\nAhead Les Glatinne 5 & shay FL Ce Lead aad Ley Aun tay sell die ly GbLi Lagan aaalls leis lay Gatially de sidall 5 ¢ gio 5 Ely GI Ste ye lgle eau + 8 gall ASacll dia Cd, le Lapa, ALLY! 5 1 Sal Gye Lede ld Ghia\n\ncL ——_ia\n\nC/T s Cpa Chath ual) (pili 9 dolby Ch Sha gla Leg yeaa!) Glia ile 1a) oda) Gs Aastall € shall y Elia) Aalgy Usenet ae g 6) dl) Le ala VeVA/V/VA plies Lae OLS Elba Jagd Vlas g ¢ AaSacal) abel aad Le clas Asti) Gludatl (ule CH dtiwall es Y 5 Legale | g3 ye oa Cpe Casall 9 Lagat jLiual) G)atival) (itedas ~ gid primal) Jats 5.2) oll\n\nASSN Gye pales apart cle Ci ghil ag V1 4/6/¥e 4S gall Catal Uae Gf allt AS jl) Aealaas uses —

In [20]:
chunks[0].metadata.orig_elements

In [21]:
elements = chunks[0].metadata.orig_elements
chunk_text = [el for el in elements if 'Text' in str(type(el))]
 

In [22]:
chunk_text

In [23]:
texts = []
for chunk in chunks:
     if "CompositeElement" in str(type((chunk))):
          texts.append(chunk)

In [24]:
texts[0].to_dict()

{'type': 'CompositeElement',
 'element_id': 'd142fd3b1c52141ef389e0d0e2a4122e',
 'text': 'ol Crom tt AU us ewe aall Gas. lS pitty (gumial! Lam ela 3S Uggle (ou GHA Sait ats gs As a: ye Aas ia gous Pig dane Slew aitase alls / duull—4\n\nVe VA/A/VV Meade Ua shall 9 £/ IS gtlae Ye VA/VAVY aby ce geall (8 4ysqll gh / sud 5 Agel) ol) die als\n\nei\n\nAhead Les Glatinne 5 & shay FL Ce Lead aad Ley Aun tay sell die ly GbLi Lagan aaalls leis lay Gatially de sidall 5 ¢ gio 5 Ely GI Ste ye lgle eau + 8 gall ASacll dia Cd, le Lapa, ALLY! 5 1 Sal Gye Lede ld Ghia\n\ncL ——_ia\n\nC/T s Cpa Chath ual) (pili 9 dolby Ch Sha gla Leg yeaa!) Glia ile 1a) oda) Gs Aastall € shall y Elia) Aalgy Usenet ae g 6) dl) Le ala VeVA/V/VA plies Lae OLS Elba Jagd Vlas g ¢ AaSacal) abel aad Le clas Asti) Gludatl (ule CH dtiwall es Y 5 Legale | g3 ye oa Cpe Casall 9 Lagat jLiual) G)atival) (itedas ~ gid primal) Jats 5.2) oll\n\nASSN Gye pales apart cle Ci ghil ag V1 4/6/¥e 4S gall Catal Uae Gf allt AS jl) Aealaas uses —

In [26]:
prompt_text = """
What is the document about?
"""
prompt = ChatPromptTemplate.from_template(prompt_text)
 
# Summary chain
model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()
summarize_chain

{
  element: RunnableLambda(lambda x: x)
}
| ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nWhat is the document about?\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x74726b5873b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x74726b58c170>, model_name='llama-3.1-8b-instant', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [27]:
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})

In [28]:
text_summaries

["It seems I don't have any information about a specific document. This conversation has just started, and I'm ready to help with any questions or topics you'd like to discuss. If you have a document you'd like to share or discuss, I'd be happy to try and assist you.",
 "This conversation has just started. I don't see any document. If you'd like to share a document or provide more context, I'd be happy to help you understand its content."]